In [1]:
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import sys
sys.path.append("..")
%cd ..

/Users/AsgerSturisTang/OneDrive - Danmarks Tekniske Universitet/DTU/6. Semester/Bachelor2021


In [33]:
from DataPrep.ImportData import importer
from Modelling import modelling
import keras
from keras import Sequential, Model
from keras.layers import Dense, LSTM, Dropout, Input, Reshape
import pyforest
import torch
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from Modelling.Modelling import modelling



ModuleNotFoundError: No module named 'Modelling.Modelling'; 'Modelling' is not a package

In [3]:
df = importer().Import()

In [10]:
df = df.set_index("Start Date")


KeyError: "None of ['Start Date'] are in the columns"

In [11]:
df.index = df.index.to_period("D")

PeriodIndex(['2011-08-03', '2011-08-04', '2011-08-05', '2011-08-06',
             '2011-08-07', '2011-08-08', '2011-08-09', '2011-08-10',
             '2011-08-11', '2011-08-12',
             ...
             '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
             '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
             '2019-12-30', '2019-12-31'],
            dtype='period[D]', name='Start Date', length=17281, freq='D')

In [5]:
df.head()

,Label,Charging Time (mins),Energy (kWh),Total Duration (mins),Port Number,CenterLon,CenterLat,Level 1,Level 2,Energy (kWh)_lag1,...,Year_Month_5,Year_Month_6,Year_Month_7,Year_Month_8,Year_Month_9,Year_Month_10,Year_Month_11,Year_Month_12,is_holiday,is_weekend
Start Date,,,,,,,,,,,,,,,,,,,,,
2011-08-03,0.0,518.816402,1.679882,712.717196,16.0,-122.16,37.445,0,8,2.098609,...,0,0,0,1,0,0,0,0,0,0
2011-08-04,0.0,405.733598,1.899242,639.499471,10.0,-122.16,37.445,0,5,1.679882,...,0,0,0,1,0,0,0,0,0,0
2011-08-05,0.0,573.500000,1.574471,592.750000,20.0,-122.16,37.445,0,10,1.899242,...,0,0,0,1,0,0,0,0,0,0
2011-08-06,0.0,897.216667,1.982032,1274.100000,28.0,-122.16,37.445,0,14,1.574471,...,0,0,0,1,0,0,0,0,0,1
2011-08-07,0.0,252.950000,2.232860,1311.750000,6.0,-122.16,37.445,0,3,1.982032,...,0,0,0,1,0,0,0,0,0,1


In [6]:
labels = df.Label.unique()

In [7]:
print(min(df.index), max(df.index))

2011-08-03 00:00:00 2019-12-31 00:00:00


In [9]:
df_reindex = pd.DataFrame()

In [16]:
idx = pd.period_range(min(df.index), max(df.index))
for i in labels:
    filling = df[df.Label == i].reindex(idx, fill_value = 0)
    filling.Label = i
    df_reindex = df_reindex.append(filling)


In [34]:
m = modelling()

In [45]:
X_train,X_test, X_val,y_train,y_test, y_val = m.ttsplit(df_reindex, shuffle = False)